## Week 10 Lecture 2 - Social Networks

McElreath's lectures for today: https://www.youtube.com/watch?v=L_QumFUv7C8&list=PLDcUM9US4XdMROZ57-OIRtIK0aOynbgZN&index=16

McElreath's lectures for the whole book are available here: https://github.com/rmcelreath/stat_rethinking_2022

An R/Stan repo of code is available here: https://vincentarelbundock.github.io/rethinking2/

Dustin Stansbury has some lovely PyMC Code available here: https://github.com/dustinstansbury/statistical-rethinking-2023

You are encouraged to work through both of these versions to re-enforce what we're doing in class.

In [ ]:
# Import python packages
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp 
import random as rd
import pdb
import pymc as pm
import arviz as az
import networkx as nx
from matplotlib import pyplot as plt
import dataframe_image as dfi
from matplotlib.patches import Ellipse, transforms


# Helper functions
def stdize(x):
    return (x-np.mean(x))/np.std(x)


def indexall(L):
    poo = []
    for p in L:
        if not p in poo:
            poo.append(p)
    Ix = np.array([poo.index(p) for p in L])
    return poo,Ix

def logit(p):
    return np.log(p) - np.log(1 - p)

def invlogit(p):
    return np.exp(p) / (1 + np.exp(p))


from matplotlib.patches import Ellipse
from scipy.stats import chi2


def Gauss2d(mu, cov, ci, ax=None, ec='k'):
    """Copied from statsmodel"""
    if ax is None:
        _, ax = plt.subplots(figsize=(6, 6))

    v_, w = np.linalg.eigh(cov)
    u = w[0] / np.linalg.norm(w[0])
    angle = np.arctan(u[1]/u[0])
    angle = 180 * angle / np.pi # convert to degrees

    for level in ci:
        v = 2 * np.sqrt(v_ * chi2.ppf(level, 2)) #get size corresponding to level
        ell = Ellipse(mu[:2], v[0], v[1], 180 + angle, facecolor='None',
                      edgecolor=ec,
                      alpha=(1-level)*.5,
                      lw=1.5)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
    
    return ax

# Social Networks

## Social relations as correlated varying effects

The data for this example consist of pairs of households (dyads) in Nicaragua, the publicaiton of which you can find in the original paper, [Koester and Leckie 2014](https://www.researchgate.net/profile/Jeremy_Koster/publication/261764179_Food_sharing_networks_in_lowland_Nicaragua_An_application_of_the_social_relations_model_to_count_data/links/5c413437299bf12be3d04539/Food-sharing-networks-in-lowland-Nicaragua-An-application-of-the-social-relations-model-to-count-data.pdf). In short, the model estimates gift giving between pairs of households ('dyads': in sociology ['a group of two people'](https://en.wikipedia.org/wiki/Dyad_(sociology))). Let's import the data and see what it looks like




In [ ]:
kdata = pd.read_csv('kl_dyads.csv')
dfi.export(kdata.head(10), 'kl_dyads.jpg')
kdata.head()

In [ ]:
plt.scatter(kdata.giftsAB, kdata.giftsBA)
plt.axline((0,0),(max(kdata.giftsBA),max(kdata.giftsBA)),linestyle=":",linewidth=3,c='black',zorder=0)
plt.savefig('gifts.jpg',dpi=300);

Basically households (`hidA` and `hidB`) trade with each other unequally (`giftsAB` and `giftsBA`) in a series of recorded exchanges between pairs (`did`). The question here is how does gift giving vary among households and among dyad pairs?

In [ ]:
# Number of observations
N = len(kdata)

# Number of households
N_households = max(kdata.hidB)

# Dyad ID
did = kdata.did.values-1
# A households - note A and B have no meaning
hidA = kdata.hidA.values-1
# B households
hidB = kdata.hidB.values-1
# Household labels
HH = ['HH_'+x for x in np.sort(np.unique(np.append(hidA,hidB))).astype(str)]

# Gifts from A to B
giftsAB = kdata.giftsAB
# Gifts from B to A
giftsBA = kdata.giftsBA

# Dyads basic model

The question here is to estiamte average giving and receiving rates for Nicaraguan households, as well as rates for particular pairs (dyads) of households. Because gifts from household A to B hold no priority over gifts from B to A, a model that does not depend on direction is required. However there are directions involved, in that a gift from A to B has a direction and is equivalent to a reception from B to A. This implies two key observation-level models:

$$
\begin{align}
y_{A\rightarrow B} \sim & Poisson(\lambda_{AB}) \\
log(\lambda_{AB}) = & \alpha + g_{A} + r_{B} + d_{AB}
\end{align}
$$

And conversely

$$
\begin{align}
y_{B\rightarrow A} \sim & Poisson(\lambda_{BA}) \\
log(\lambda_{BA}) = & \alpha + g_{B} + r_{A} + d_{BA}
\end{align}
$$

where $\alpha$ represents an average rate of giving; $g_{A}$ represents the average rate of giving from household A, $r_{B}$ represents the average rate of receiving for household B, and $d_{AB}$ is the dyad-specific giving rate from A to B. Doing this implies that each household H needs varying effects for giving $g_{H}$ and receiving $r_{H}$. On top of this each dyad has their own specific $d_{AB}$ and $d_{BA}$ effects. One goal here is to see if the $g$ and $r$ paremters are correlated - go givers get a lot in return? In addition, we're interested in knowing if there are asymetries in gifts within each dyad - is there a gift balance or not for each pair? These are handled with a couple of multi-normal priors. The first deals with the household effects:

$$
\begin{pmatrix} g_{i} \\ r_{i} \end{pmatrix} \sim MvN \left( \begin{pmatrix} 0 \\ 0 \end{pmatrix}, \begin{pmatrix} \sigma_{g}^2 & \sigma_{g}\sigma_{r}\rho_{gr} \\ \sigma_{g}\sigma_{r}\rho_{gr} & \sigma_{r}^2 \end{pmatrix} \right).
$$

The second, deals with the dyad effects:

$$
\begin{pmatrix} d_{ij} \\ d_{ji} \end{pmatrix} \sim MvN \left( \begin{pmatrix} 0 \\ 0 \end{pmatrix}, \begin{pmatrix} \sigma_{d}^2 & \sigma_{d}^2\rho_{d} \\ \sigma_{d}^2\rho_{d} & \sigma_{d}^2 \end{pmatrix} \right).
$$

It's important to note that the standard deviation $\sigma_{d}$ is common for both terms because the direction (i or j) doesn't matter. And $\rho_{d}$ tells us if there is an asymetry ($\rho_{d}$ negative) or not ($\rho_{d}$ positive). 

We can implement this in pymc as:

In [ ]:
with pm.Model(coords={"Rate": ["giving", "receiving"], "HouseH":HH}) as Dyads:
    # gr matrix of varying effects per household
    sd_dist = pm.Exponential.dist(1.0)
    chol_gr, _, _ = pm.LKJCholeskyCov("chol_gr", n=2, eta=4, sd_dist=sd_dist, compute_corr=True)
    gr = pm.MvNormal("gr", mu=0, chol=chol_gr, shape=(N_households, 2), dims=('HouseH','Rate'))

    # dyad effects
    chol_dyad, _, _ = pm.LKJCholeskyCov("chol_dyad", n=2, eta=8, sd_dist=sd_dist, compute_corr=True)
    z = pm.Normal("z", 0, 1, shape=(2, N))
    d = pm.Deterministic("d", pm.math.dot(chol_dyad, z).T)

    # linear models
    a = pm.Normal("a", 0, 1)
    lambdaAB = pm.math.exp(a + gr[hidA, 0] + gr[hidB, 1] + d[did, 0])
    lambdaBA = pm.math.exp(a + gr[hidB, 0] + gr[hidA, 1] + d[did, 1])

    # likelihood
    YAB_ = pm.Poisson("giftsAB", lambdaAB, observed=giftsAB)
    YBA = pm.Poisson("giftsBA", lambdaBA, observed=giftsBA)

In [ ]:
with Dyads:
    trace = pm.sample()

In [ ]:
Dyadz = trace.copy().rename_dims({"d": ["Dyad", "House"], "gr": ["Household", "Rate"]})

In [ ]:
# Rename sub-objects within chol_gr and chol_dyad
PostDyads = Dyadz.posterior = Dyadz.posterior.rename_vars({"chol_gr_corr": "Rho_gr", "chol_gr_stds": "sigma_gr", "chol_dyad_corr": "Rho_d", "chol_dyad_stds": "sigma_d"})

In [ ]:
tmp = az.summary(Dyadz, var_names=["Rho_gr", "sigma_gr"], round_to=2)
dfi.export(tmp, 'household_corr.jpg')
tmp

The correlation here is -0.41, which implies that individuals who give more thend to receive less across all dyads. The standard deviation for giving (0.83) is twice as varaible as rates of receiving (0.41).

Let's take a look at the estimated household giving and receiving rates

In [ ]:
# Household level log-giving rate posteriors
g = (PostDyads["a"] + PostDyads["gr"].sel(Rate="giving")).stack(sample=("chain", "draw"))
# Household level log-receiving rate posteriors
r = (PostDyads["a"] + PostDyads["gr"].sel(Rate="receiving")).stack(sample=("chain", "draw"))

# Household expected giving rates
Eg_mu = np.exp(g).mean(dim="sample")
# Household expected receiving rates
Er_mu = np.exp(r).mean(dim="sample")

In [ ]:
_, ax = plt.subplots(1, 1, constrained_layout=True)
x = np.linspace(0, 9, 101)
ax.plot(x, x, "k--", lw=1.5, alpha=0.4)

# Plot uncertainty ellipses
for house in range(25):
    Sigma = np.cov(np.stack([np.exp(g[house].values), np.exp(r[house].values)]))
    Mu = np.stack([np.exp(g[house].values.mean()), np.exp(r[house].values.mean())])
    pearson = Sigma[0, 1] / np.sqrt(Sigma[0, 0] * Sigma[1, 1])
    ellipse = Ellipse((0, 0),np.sqrt(1 + pearson),np.sqrt(1 - pearson),edgecolor="k",alpha=0.5,facecolor="none",)
    std_dev = sp.stats.norm.ppf((1 + np.sqrt(0.5)) / 2)
    scale_x = 2 * std_dev * np.sqrt(Sigma[0, 0])
    scale_y = 2 * std_dev * np.sqrt(Sigma[1, 1])
    scale = transforms.Affine2D().rotate_deg(45).scale(scale_x, scale_y)
    ellipse.set_transform(scale.translate(Mu[0], Mu[1]) + ax.transData)
    ax.add_patch(ellipse)

# household means
ax.plot(Eg_mu, Er_mu, "ko", mfc="white", lw=1.5)

ax.set(xlim=(0, 8.6),ylim=(0, 8.6),xlabel="generalized giving",ylabel="generalized receiving",)
plt.savefig('exchange.jpg',dpi=300);

What becomes clear in this is that there are a bunch of generous households that receive very little and that there are some households that don't give much but often receive more. 

At the dyad level, what do the paired gift exchange relationships look like?

In [ ]:
tmp = az.summary(Dyadz, var_names=["Rho_d", "sigma_d"], round_to=2)
dfi.export(tmp, 'dyad_corr.jpg')
tmp

In [ ]:
# Grab dyad giving
dy1 = PostDyads["d"].mean(dim=("chain", "draw")).T[0]
dy2 = PostDyads["d"].mean(dim=("chain", "draw")).T[1]

In [ ]:
_, ax = plt.subplots(1, 1, constrained_layout=True)
x = np.linspace(-2, 4, 101)

ax.plot(x, x, "k--", lw=1.5, alpha=0.4)
ax.axhline(linewidth=1.5, color="k", ls="--", alpha=0.4)
ax.axvline(linewidth=1.5, color="k", ls="--", alpha=0.4)
ax.plot(dy1, dy2, "ko", mfc="none", lw=1.5, alpha=0.6)

ax.set(xlim=(-2, 4),ylim=(-2, 4),xlabel="household A in dyad",ylabel="household B in dyad",)
plt.savefig('exchange2.jpg',dpi=300);

Here we can see that, once we have accounted for individual household behaviour, there is a remarkable consistency within dyads - balanced giving.

## Covariates

Looking at the households data and thinking about potential giving relationships, we can readily think of four potential relationships:

- Average giving
- Generous households
- Needy households
- Family linkages (specific dyads)

The last, family linkages, is represented in the `drel`1-4 colums in the data


In [ ]:
kdata.head()

We aslo have other covariate information at the household level from the researchers

In [ ]:
hdata = pd.read_csv('kl_households.csv')
dfi.export(hdata.head(), 'hh_cov.jpg')
hdata.head()

Let's follow the lead from the original paper and incorporate all these into our model. First grab the covariates

In [ ]:
# Household ID
hid = hdata.hid.values

# kg meat harvested per day
hgame = stdize(hdata.hgame.values)
# kg fish harvested per day
hfish = stdize(hdata.hfish.values)
# avg number of pigs owned during study
hpigs = stdize(hdata.hpigs.values)
# household weath index
hwealth = stdize(hdata.hwealth.values)
# Pastor in receiving household
hpastor = stdize(hdata.hpastor.values)

In [ ]:
# Number of observations
N = len(kdata)
# Number of households
N_households = max(kdata.hidB)
# Dyad ID
Id = kdata.did.values-1
# Index household
IhA = kdata.hidA.values-1
# Index household
IhB = kdata.hidB.values-1
# Gifts from A to B
giftsAB = kdata.giftsAB
# Gifts from B to A
giftsBA = kdata.giftsBA

# Mother-offspring (r=0.5)
drel1 = kdata.drel1.values
# Father-offspring OR sibling (r=0.5)
drel2 = kdata.drel2.values
# Close relative (0.25<r<0.5)
drel3 = kdata.drel3.values
# Distant relative (0.1<r<0.25)
drel4 = kdata.drel4.values

# Distance between households on log-scale (km)
dlndist = stdize(kdata.dlndist.values)

# Frequency of association 
dass = stdize(kdata.dass.values)

# ?
d0125 = stdize(kdata.d0125.values)

# Offset?
oset = kdata.offset.values

# Dyads covariate model

Buidling on the intercept-only model, we can look at Koester and Leicke's model, written in `WinBUGS`:

```
model{

  #Dyadic response distributions
  for(d in 1:300) {
  
    # Observed gifts from A to B modeled as Poisson distributed
    giftsAB[d] ~ dpois(muAB[d])
    
    # Observed gifts from B to A modeled as Poisson distributed
    giftsBA[d] ~ dpois(muBA[d])

    # Linear predictor for log of expected gifts from A to B
    log(muAB[d]) <- offset[d] 
                  + beta[1]
                  + beta[2]*hgame[hidA[d]]
                  + beta[3]*hfish[hidA[d]]
                  + beta[4]*hpigs[hidA[d]]
                  + beta[5]*hwealth[hidA[d]]
                  + beta[6]*hgame[hidB[d]]
                  + beta[7]*hfish[hidB[d]]
                  + beta[8]*hpigs[hidB[d]]
                  + beta[9]*hwealth[hidB[d]]
                  + beta[10]*hpastor[hidB[d]]
                  + beta[11]*drel1[d]
                  + beta[12]*drel2[d]
                  + beta[13]*drel3[d]
                  + beta[14]*drel4[d]
                  + beta[15]*dlndist[d]
                  + beta[16]*dass[d] 
                  + beta[17]*d0125[d] 
                  + gr[hidA[d],1] + gr[hidB[d],2] + dd[d,1]
    
    # Linear predictor for log of expected gifts from A to B
    log(muBA[d]) <- offset[d] 
                  + beta[1]  
                  + beta[2]*hgame[hidB[d]]
                  + beta[3]*hfish[hidB[d]]
                  + beta[4]*hpigs[hidB[d]]
                  + beta[5]*hwealth[hidB[d]]
                  + beta[6]*hgame[hidA[d]]
                  + beta[7]*hfish[hidA[d]]
                  + beta[8]*hpigs[hidA[d]]
                  + beta[9]*hwealth[hidA[d]]
                  + beta[10]*hpastor[hidA[d]]
                  + beta[11]*drel1[d]
                  + beta[12]*drel2[d]
                  + beta[13]*drel3[d]
                  + beta[14]*drel4[d]
                  + beta[15]*dlndist[d]
                  + beta[16]*dass[d] 
                  + beta[17]*d0125[d]
                  + gr[hidB[d],1] + gr[hidA[d],2] + dd[d,2]
    
  }


  #Giver and receiver bivariate normal random effects
  for (h in 1:25) {
    gr[h,1:2] ~ dmnorm(zero[1:2],TAU_gr[1:2,1:2])
  }
  zero[1] <- 0
  zero[2] <- 0


  #Relationship bivariate normal random effects
  for(d in 1:300) {
    dd[d,1:2] ~ dmnorm(zero[1:2],TAU_dd[1:2,1:2])
  }

  #Priors for fixed effects regression coefficients
  for (k in 1:17) {
    beta[k] ~ dflat()
  }


  #Priors for giver-receiver variance-covariance matrix
  TAU_gr[1:2,1:2] ~ dwish(R_gr[1:2,1:2],2)
  COV_gr[1:2,1:2] <- inverse(TAU_gr[,])


  #Priors for relationship variance-covariance matrix  
  TAU_dd[1:2,1:2] <- inverse(COV_dd[1:2,1:2])
  COV_dd[1,1] <- sigma2_d
  COV_dd[1,2] <- sigma_dd
  COV_dd[2,1] <- sigma_dd
  COV_dd[2,2] <- sigma2_d
  sigma_dd <- rho_dd*sigma2_d
  sigma2_d <- 1/tau_d
  tau_d ~ dgamma(0.001,0.001)
  rho_dd ~ dunif(-1,1)

}
```

In [ ]:
with pm.Model(coords={"Rate": ["giving", "receiving"], "HouseH":HH}) as DyadsCOV:
    # gr matrix of varying effects per household
    sd_dist = pm.Exponential.dist(1.0)
    chol_gr, _, _ = pm.LKJCholeskyCov("chol_gr", n=2, eta=4, sd_dist=sd_dist, compute_corr=True)
    gr = pm.MvNormal("gr", mu=0, chol=chol_gr, shape=(N_households, 2), dims=('HouseH','Rate'))

    # dyad effects
    chol_dyad, _, _ = pm.LKJCholeskyCov("chol_dyad", n=2, eta=8, sd_dist=sd_dist, compute_corr=True)
    z = pm.Normal("z", 0, 1, shape=(2, N))
    d = pm.Deterministic("d", pm.math.dot(chol_dyad, z).T)

    # Household effects
    a = pm.Normal("intercept", 0, 1)
    b2 = pm.Normal("game_g", 0, 1)
    b3 = pm.Normal("fish_g", 0, 1)
    b4 = pm.Normal("pigs_g", 0, 1)
    b5 = pm.Normal("wealth_g", 0, 1)
    b6 = pm.Normal("game_r", 0, 1)
    b7 = pm.Normal("fish_r", 0, 1)
    b8 = pm.Normal("pigs_r", 0, 1)
    b9 = pm.Normal("wealth_r", 0, 1)
    b10 = pm.Normal("pastor_r", 0, 1)
    b11 = pm.Normal("rel1", 0, 1)
    b12 = pm.Normal("rel2", 0, 1)
    b13 = pm.Normal("rel3", 0, 1)
    b14 = pm.Normal("rel4", 0, 1)
    b15 = pm.Normal("distance", 0, 1)
    b16 = pm.Normal("association", 0, 1)
    b17 = pm.Normal("d0125", 0, 1)
    
    # linear models
    lambdaAB = pm.math.exp(oset + a + b2*hgame[IhA] + b3*hfish[IhA] + b4*hpigs[IhA] + b5*hwealth[IhA] + 
                      b6*hgame[IhB] + b7*hfish[IhB] + b8*hpigs[IhB] + b9*hwealth[IhB] + 
                      b10*hpastor[IhB] + b11*drel1 + b12*drel2 + b13*drel3 + b14*drel4 + b15*dlndist + b16*dass + 
                      b17*d0125 + gr[IhA, 0] + gr[IhB, 1] + d[Id, 0])
    lambdaBA = pm.math.exp(oset + a + b2*hgame[IhB] + b3*hfish[IhB] + b4*hpigs[IhB] + b5*hwealth[IhB] + 
                      b6*hgame[IhA] + b7*hfish[IhA] + b8*hpigs[IhA] + b9*hwealth[IhA] + 
                      b10*hpastor[IhA] + b11*drel1 + b12*drel2 + b13*drel3 + b14*drel4 + b15*dlndist + b16*dass + 
                      b17*d0125 + gr[IhB, 0] + gr[IhA, 1] + d[Id, 1])

    # likelihood
    YgiftsAB = pm.Poisson("YgiftsAB", lambdaAB, observed=giftsAB)
    YgiftsBA = pm.Poisson("YgiftsBA", lambdaBA, observed=giftsBA)

In [ ]:
with DyadsCOV:
    trace_c = pm.sample()

In [ ]:
Dyadz = trace_c.copy().rename_dims({"d": ["Dyad", "House"], "gr": ["Household", "Rate"]})

In [ ]:
# Rename sub-objects within chol_gr and chol_dyad
PostDyads = Dyadz.posterior = Dyadz.posterior.rename_vars({"chol_gr_corr": "Rho_gr", "chol_gr_stds": "sigma_gr", "chol_dyad_corr": "Rho_d", "chol_dyad_stds": "sigma_d"})

In [ ]:
tmp = az.summary(Dyadz, var_names=["Rho_gr", "sigma_gr"], round_to=2)
dfi.export(tmp, 'household_corr_cov.jpg')
tmp

The correlation here was -0.41 in the non-covariate model (which implied that individuals who give more thend to receive less across all dyads) but is now down to 0.05, implying that there is no remaining correlation between givers and receivers given the covariates in the model. The standard deviation for giving is now down to 0.5 (from 0.83) and the variation in rates of receiving is down to 0.3 (from 0.41).

Let's take a look at the estimated household giving and receiving rates and how they have changed

In [ ]:
# Household level log-giving rate posteriors
g = (PostDyads["intercept"] + PostDyads["gr"].sel(Rate="giving")).stack(sample=("chain", "draw"))
# Household level log-receiving rate posteriors
r = (PostDyads["intercept"] + PostDyads["gr"].sel(Rate="receiving")).stack(sample=("chain", "draw"))

# Household expected giving rates
Eg_mu = np.exp(g).mean(dim="sample")
# Household expected receiving rates
Er_mu = np.exp(r).mean(dim="sample")

In [ ]:
_, ax = plt.subplots(1, 1, constrained_layout=True)
x = np.linspace(0, 9, 101)
ax.plot(x, x, "k--", lw=1.5, alpha=0.4)

# Plot uncertainty ellipses
for house in range(25):
    Sigma = np.cov(np.stack([np.exp(g[house].values), np.exp(r[house].values)]))
    Mu = np.stack([np.exp(g[house].values.mean()), np.exp(r[house].values.mean())])
    pearson = Sigma[0, 1] / np.sqrt(Sigma[0, 0] * Sigma[1, 1])
    ellipse = Ellipse((0, 0),np.sqrt(1 + pearson),np.sqrt(1 - pearson),edgecolor="k",alpha=0.5,facecolor="none",)
    std_dev = sp.stats.norm.ppf((1 + np.sqrt(0.5)) / 2)
    scale_x = 2 * std_dev * np.sqrt(Sigma[0, 0])
    scale_y = 2 * std_dev * np.sqrt(Sigma[1, 1])
    scale = transforms.Affine2D().rotate_deg(45).scale(scale_x, scale_y)
    ellipse.set_transform(scale.translate(Mu[0], Mu[1]) + ax.transData)
    ax.add_patch(ellipse)

# household means
ax.plot(Eg_mu, Er_mu, "ko", mfc="white", lw=1.5)

ax.set(xlim=(0, 8.6),ylim=(0, 8.6),xlabel="generalized giving",ylabel="generalized receiving",)
plt.savefig('exchange_cov.jpg',dpi=300);

The previous strong disparities between givers and receivers have been shrunk down, suggesting again that the covariates explain these very generous or needy households


At the dyad level, what do the paired gift exchange relationships look like?

In [ ]:
tmp = az.summary(Dyadz, var_names=["Rho_d", "sigma_d"], round_to=2)
dfi.export(tmp, 'dyad_corr_cov.jpg')
tmp

Here the remaining correlaiton for equal giving has shrunk from 0.88 to 0.5, expressing that the covariates also account for some of these effects.

In [ ]:
# Grab dyad giving
dy1 = PostDyads["d"].mean(dim=("chain", "draw")).T[0]
dy2 = PostDyads["d"].mean(dim=("chain", "draw")).T[1]

In [ ]:
_, ax = plt.subplots(1, 1, constrained_layout=True)
x = np.linspace(-2, 4, 101)

ax.plot(x, x, "k--", lw=1.5, alpha=0.4)
ax.axhline(linewidth=1.5, color="k", ls="--", alpha=0.4)
ax.axvline(linewidth=1.5, color="k", ls="--", alpha=0.4)
ax.plot(dy1, dy2, "ko", mfc="none", lw=1.5, alpha=0.6)

ax.set(xlim=(-2, 4),ylim=(-2, 4),xlabel="household A in dyad",ylabel="household B in dyad",)
plt.savefig('exchange2_cov.jpg',dpi=300);

Which becomes clear here in that the extremes have been reduced.

Which variables might these effects be attributable to (keeping in mind this is pure causal salad): 



In [ ]:
pm.plot_forest(trace_c, var_names=['intercept','game_g','fish_g','pigs_g','wealth_g','game_r','fish_r','pigs_r','wealth_r','pastor_r','rel1','rel2','rel3','rel4','distance','association','d0125'],figsize=(8, 8))
plt.axvline(0)
plt.tight_layout()
plt.savefig('covariate_forest.jpg',dpi=300);